In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from collections import Counter
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
lemm = WordNetLemmatizer()

lines=100000





In [2]:
def create_lexicon():
    lexicon=[]
    with open('pos.txt','r') as f:
        contents = f.readlines()
        #all_words=word_tokenize(contents)
        for l in contents[:lines]:
            all_words=word_tokenize(l)
            lexicon+=list(all_words)

                
                    
            
    with open('neg.txt','r') as f:
        contents = f.readlines()
        for l in contents[:lines]:
            all_words=word_tokenize(l)
            lexicon += list(all_words)
    
    lexicon = [lemm.lemmatize(i,pos='n') for i in lexicon]
    w_counts = Counter(lexicon)
    l2 = []
    for w in w_counts:
        #print(w_counts[w])
        if 1000 > w_counts[w] > 50:
            l2.append(w)
    return l2
            
        
            
           

In [3]:
l2=create_lexicon()

In [4]:
print((len(l2)))

423


In [5]:
def make_features(sample,lexicon,char):
    import numpy as np
    featureset=[]
    with open(sample,'r')as f:
        contents=f.readlines()
        
        for l in contents[:lines]:
            sam_words=word_tokenize(l)
            sam_words=[lemm.lemmatize(i) for i in sam_words]
            features=np.zeros(len(lexicon))
            for word in sam_words:
                if word.lower() in lexicon:
                    index_value=lexicon.index(word.lower())
                    features[index_value]+=1
            features=list(features)
            featureset.append([features,char])
    return featureset
                    
                    
            
            
              
              
    
    

In [6]:
l=make_features('pos.txt',l2,[1,0])


In [7]:
l+=make_features('neg.txt',l2,[0,1])

In [8]:
print((len(l)))

10662


In [9]:
import random
random.shuffle(l)


In [10]:
import numpy as np
features=np.array(l,dtype=list)
print(features.shape)


(10662, 2)


In [11]:
test_size=2162
a=features.shape[0]
train_size=a-test_size
x_train=features[0:train_size,0]
y_train=features[0:train_size,1]
x_test=features[train_size:a,0]
y_test=features[train_size:a,1]
print((y_test.shape))


(2162,)


In [12]:
x_train=np.vstack([np.expand_dims(x,0) for x in x_train])
y_train=np.vstack([np.expand_dims(x,0) for x in y_train])
x_test=np.vstack([np.expand_dims(x,0) for x in x_test])
y_test=np.vstack([np.expand_dims(x,0) for x in y_test])

In [13]:
print(y_test.shape)

(2162, 2)


In [26]:
h1=200
h2=200
h3=200
import tensorflow as tf

x=tf.placeholder(shape=[None,423],dtype=tf.float32)
y=tf.placeholder(shape=[None,2],dtype=tf.float32)
def forward(data):
    a=len(x_train[0])
    std1=(1/np.sqrt(a))
    w1=tf.Variable(np.random.normal(0,std1,(a,h1)))
    b1=tf.Variable(tf.constant(0.2,shape=[h1]))
    w1=tf.cast(w1,tf.float32)
    
    std2=(1/np.sqrt(h1))
    w2=tf.Variable(np.random.normal(0,std2,(h1,h2)))
    b2=tf.Variable(tf.constant(0.2,shape=[h2]))
    w2=tf.cast(w2,tf.float32)
    
    std3=(1/np.sqrt(h2))
    w3=tf.Variable(np.random.normal(0,std3,(h2,2)))
    b3=tf.Variable(tf.constant(0.2,shape=[2]))
    w3=tf.cast(w3,tf.float32)
    
    a0=data
    a1=tf.nn.relu(tf.matmul(a0,w1) +b1)
    a2=tf.nn.relu(tf.matmul(a1,w2) +b2)
    a3=(tf.matmul(a2,w3) +b3)
    #a4=tf.nn.softmax(tf.matmul(a3,w4) +b4)
    return a3
def train():
    batch=20
    #h=20
    pred=forward(x)
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
    train_step = tf.train.AdamOptimizer(0.01).minimize(cost)
    correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for j in range(8):
            for i in range(425):
                
                x_batch=x_train[i*batch:(i+1)*batch,:]
                y_batch=y_train[i*batch:(i+1)*batch,:]
                sess.run(train_step,feed_dict={x:x_batch,y:y_batch})
                
                

            #print('accuracy after epoch is',sess.run(accuracy,feed_dict={x:x_train,y:y_train}))
            print('corresponding loss',sess.run(cost,feed_dict={x:x_train,y:y_train}))
        
        
        print('accuracy for test set',sess.run(accuracy,feed_dict={x:x_test,y:y_test}))